In [3]:
# Load scikit's random forest classifier library
from sklearn.ensemble import RandomForestClassifier

# Load pandas
import pandas as pd

# Load numpy
import numpy as np

# Set random seed
np.random.seed(0)

In [4]:
from collections import defaultdict
import math

In [5]:
reader=pd.read_csv('F://topical.txt',delimiter="\t",header=None,names=["business_id","business_stars","business_review_count","review_id","review_stars","text","user_id","name","user_review_count","yelping_since","friends","useful","funny","cool","fans","elite"],encoding='latin-1')

In [51]:
#reader.head()

In [6]:
f=defaultdict(dict)
for i in range(len(reader)):
    f[reader["user_id"][i]]={};
    

In [7]:
print(f[reader["user_id"][2]])
print(len(f))
print(len(reader))
#reader=reader[:5]

{}
332716
2347774


In [8]:
for i in range(len(reader)):
    if(i%50000==0):
        print(i)
    if "review_count" in f[reader["user_id"][i]]:
        f[reader["user_id"][i]]["review_count"]+=1
    else:
        f[reader["user_id"][i]]["review_count"]=1
        
    if "ratings" in f[reader["user_id"][i]]:
        f[reader["user_id"][i]]["ratings"].append(reader["review_stars"][i])
    else:
        f[reader["user_id"][i]]["ratings"]=[]
        f[reader["user_id"][i]]["ratings"].append(reader["review_stars"][i])
        
    if "business_reviewed" in f[reader["user_id"][i]]:
        f[reader["user_id"][i]]["business_reviewed"][reader["business_id"][i]]=1
    else:
        #print("business_reviewed")
        f[reader["user_id"][i]]["business_reviewed"]={}
        f[reader["user_id"][i]]["business_reviewed"][reader["business_id"][i]]=1
        
    if "yelping_age" not in f[reader["user_id"][i]]:
#         if(type(reader["yelping_since"][i])==float and math.isnan(reader["yelping_since"][i])):
#             f[reader["user_id"][i]]["yelping_age"]=0
#         else:
#             print("Failed: ",i)
#             f[reader["user_id"][i]]["yelping_age"]= 2018-int(reader["yelping_since"][i][0:4])
        try:
            f[reader["user_id"][i]]["yelping_age"]= 2018-int(reader["yelping_since"][i][0:4])
        except:
            f[reader["user_id"][i]]["yelping_age"]=0
    #print(reader["user_id"][i],": ",f[reader["user_id"][i]])
    
        

0
50000
100000
150000
200000
250000
300000
350000
400000
450000
500000
550000
600000
650000
700000
750000
800000
850000
900000
950000
1000000
1050000
1100000
1150000
1200000
1250000
1300000
1350000
1400000
1450000
1500000
1550000
1600000
1650000
1700000
1750000
1800000
1850000
1900000
1950000
2000000
2050000
2100000
2150000
2200000
2250000
2300000


In [29]:
avg_yelp_age=0;
count=0
for key in f:
    if("yelping_age" in f[key] and f[key]["yelping_age"] !=0):
        avg_yelp_age+=f[key]["yelping_age"]
        count+=1
avg_yelp_age=avg_yelp_age/count;

count=0
remove_keys=[]
for key in f:
    try:
        if(type(key)==float and math.isnan(key)):
            continue
#         print(f[key]["ratings"])
#         print(key)
        rate=np.array(list(map(int,f[key]["ratings"])))
    #     print(f[key]["ratings"])
    #     print(key)
        avg=np.mean(rate)
        std=np.std(rate)
        f[key]["avg_rating"]=avg
        f[key]["stdev"]=std
        f[key]["unique_business_count"]=len(f[key]["business_reviewed"])
        if(f[key]["yelping_age"] ==0):
            f[key]["yelping_age"] =avg_yelp_age
        count+=1
        if(count%50000==0):
            print(f[key]["avg_rating"]," ",f[key]["stdev"]," ",f[key]["unique_business_count"])
    except:
        remove_keys.append(key)
    

1.5   0.5   2
4.0   0.0   1
5.0   0.0   1
5.0   0.0   1
3.0   0.0   1
3.0   0.0   1


In [31]:
# print(f["-3i9bhfvrM3F1wsC9XIB8g"])
# print(f["-3i9bhfvrM3F1wsC9XIB8g"]["ratings"])
# print(len(remove_keys))

for key in remove_keys:
    f.pop(key,None)

{'review_count': 1, 'ratings': ['4'], 'business_reviewed': {'GtHu9uGXpn7Jg_Z7vFKZKw': 1}, 'yelping_age': 13, 'avg_rating': 4.0, 'stdev': 0.0, 'unique_business_count': 1}
['4']
2818


In [32]:
# print(type(reader["yelping_since"][39721]))
print(f["-3i9bhfvrM3F1wsC9XIB8g"])

{'review_count': 1, 'ratings': ['4'], 'business_reviewed': {'GtHu9uGXpn7Jg_Z7vFKZKw': 1}, 'yelping_age': 13, 'avg_rating': 4.0, 'stdev': 0.0, 'unique_business_count': 1}


In [57]:
#Creating Y values
y_map={}
for i in range(len(reader)):
    if(reader["elite"][i]=="None"):
        y_map[reader["user_id"][i]]=0
    else:
        y_map[reader["user_id"][i]]=1

In [58]:
#creating feature vector
feature=[]
count=0
for key in f:
    try:
        temp=[]
        temp.append(key)
        temp.append(f[key]["review_count"])
        temp.append(f[key]["yelping_age"])
        temp.append(f[key]["avg_rating"])
        temp.append(f[key]["stdev"])
        temp.append(f[key]["unique_business_count"])
        temp.append(y_map[key])
        feature.append(temp)
        if(y_map[key]==0):
            count+=1
    except:
        print(key)
print(count)

nan
307059


In [60]:
#print(len(feature))
feature_df=pd.DataFrame(feature,columns=["user_id","review_count","yelping_age","avg_rating","stdev","unique_business_count","y"])

In [59]:
print(reader['elite'][0]=="None")
print(len(feature))

True
329897


In [62]:
feature_df['is_train']=np.random.uniform(0,1,len(feature_df)) <=0.75
#reader.head()

In [63]:
train=feature_df[feature_df['is_train']==True]
test=feature_df[feature_df['is_train']==False]
print(len(train))
print(len(test))
print(len(train)/(len(train)+len(test)))

247195
82702
0.7493096330066658


In [71]:
features=feature_df.columns[1:-2]
features
#print(train[features])
y=train["y"]
#print(y)

1         0
2         1
3         0
4         0
5         0
6         0
7         1
9         0
10        0
12        0
13        0
15        0
16        0
17        0
20        0
22        0
23        0
25        0
26        0
27        0
28        0
29        1
31        1
32        0
33        0
34        0
35        0
36        0
37        1
38        0
         ..
329862    0
329863    0
329866    0
329867    0
329868    1
329869    0
329870    0
329871    0
329872    0
329873    0
329874    0
329875    0
329876    0
329877    0
329878    1
329880    0
329881    0
329883    0
329884    0
329885    0
329886    0
329887    0
329888    0
329889    0
329890    0
329891    0
329893    0
329894    0
329895    0
329896    0
Name: y, Length: 247195, dtype: int64


In [72]:
clf=RandomForestClassifier(n_jobs=2,random_state=0)
clf.fit(train[features],y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=2,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [74]:
test_y=clf.predict(test[features])

In [84]:
count=0
count_1=0
count_actual_1=0
y_given=list(test["y"])
print(type(y_given))
for i in range(len(test)):
    if(y_given[i]==test_y[i]):
        count+=1
    if(y_given[i]==test_y[i] and test_y[i]==1):
        count_1+=1
    if(y_given[i]==1):
        count_actual_1+=1
print(count/len(test))
print(count_1/(count_actual_1))
print(count_1)
print(count_actual_1)
    

<class 'list'>
0.9291552804043433
0.08908145580589255
514
5770
